# Create LM

In [1]:
import httpx
from typing import List, Optional
import brokit as bk
class Llama(bk.LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=bk.ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[bk.Message]]=None, images:Optional[List[bk.Image]]=None, audios=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, bk.Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        if images:
            _messages[-1]["images"] = [image.to_base64() for image in images]
        request_params = {
            "model": self.model_name,
            "messages": _messages,
            "stream": False,
            "options": {**params},
        }
        response = self.client.post(
            url,
            json=request_params
        )                
        return response.json()

    def parse_response(self, original_response: dict) -> bk.ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return bk.ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=bk.Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

# Test LM with normal prompt

In [2]:
llama = Llama(model_name="gemma3:12b")

In [3]:
llama.request(prompt="Hello, how are you?")

{'model': 'gemma3:12b',
 'created_at': '2026-02-07T06:12:15.4328527Z',
 'message': {'role': 'assistant',
  'content': "Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to assist you. 😊 \n\nHow are *you* doing today?"},
 'done': True,
 'done_reason': 'stop',
 'total_duration': 3314625500,
 'load_duration': 2540963000,
 'prompt_eval_count': 15,
 'prompt_eval_duration': 55795500,
 'eval_count': 51,
 'eval_duration': 673059900}

In [4]:
response = llama(prompt="Hello, how are you?")
response.cached, response

(False,
 ModelResponse(model_name='gemma3:12b', model_type=<ModelType.CHAT: 'chat'>, response="Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to help. 😊\n\nHow are *you* doing today?", usage=Usage(input_tokens=15, output_tokens=49), response_ms=916.8049999861978, cached=False, metadata=None, request=None, parsed_response=None))

In [5]:
resopnse = llama(prompt="Hello, how are you?")
response.cached, response

(True,
 ModelResponse(model_name='gemma3:12b', model_type=<ModelType.CHAT: 'chat'>, response="Hello! I'm doing well, thank you for asking! As an AI, I don't experience feelings like humans do, but I'm operating smoothly and ready to help. 😊\n\nHow are *you* doing today?", usage=Usage(input_tokens=15, output_tokens=49), response_ms=916.8049999861978, cached=True, metadata=None, request=None, parsed_response=None))

# Test LM with Image

In [6]:
doraemon1 = bk.Image("https://aithailand.co.th/wp-content/uploads/2024/09/Doraemon-th_1630640671-1024x635.jpg")
nobita1 = bk.Image("https://m.media-amazon.com/images/M/MV5BM2E5MDIwNjktOWQwYS00ODg5LTlhN2MtNzU1OTc3MDA0NjNmXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg")
doraemon1.to_base64() == nobita1.to_base64()

False

In [7]:
llama = Llama(model_name="gemma3:12b")
response = llama(prompt="What's in the image??", images=[doraemon1, nobita1])
print(response.response)

Here's a breakdown of what's in the images:

**Image 1:**

*   **Character:** It's Doraemon, a popular Japanese manga and anime character. He's a robotic cat from the future.
*   **Appearance:** He's blue, has a red nose, big eyes, and a bell around his neck. He's waving.
*   **Text:** The word "DORAEMON" is written below his image.
*   **Background:** A light blue background with a white border.

**Image 2:**

*   **Character:** This is Nobita Nobi, another character from the Doraemon series. He's the main protagonist.
*   **Appearance:** He's a young boy with black hair, glasses, and a slightly mischievous expression.
*   **Background:** A blue sky with a green tree and a roof.



Let me know if you'd like to know more about Doraemon or Nobita!


In [8]:
model_name = "gemma3:12b" # this model can take more than one image at a time
# model_name = "llama3.2-vision:11b" # this model can take only one image at a time
llama = Llama(model_name=model_name)
prompt = "Who are they from cartoon, Doraemon?" # if we give some hint, describing images is suddenly an easy task
# response = llama(prompt=prompt, images=[nobita1])
# response = llama(prompt=prompt, images=[doraemon1])
response = llama(prompt=prompt, images=[doraemon1, nobita1])
# response = llama(prompt=prompt, images=[nobita1, doraemon1])
print(response.response)

Certainly! Let's identify the characters from the cartoon Doraemon.

**Image 1:**

*   This is **Doraemon** himself! He's the main character of the series. He's a robotic cat from the future sent back in time to help a young boy named Nobita.

**Image 2:**

*   This is **Nobita Nobi**. He's the boy that Doraemon is sent to help. He's known for being clumsy and often needing Doraemon's gadgets to get out of trouble.

Doraemon is a very popular and beloved Japanese manga and anime series!



Let me know if you'd like to know more about the characters or the show!


In [9]:
llama.history

[]